<a href="https://colab.research.google.com/github/yelim421/SolarPower/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Solar_Power

/content/drive/MyDrive/Solar_Power


In [5]:
import itertools
import tensorflow as tf
from keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
with open('done_preprocessing.csv', 'r') as f:
    data = pd.read_csv(f)

In [7]:
data.head()

,aerosol,year,month,day,hour,Temp(C),Prec(mm),Humidity(%),Dew_Point(C),Solar_Radiation(MJ/m2),Snowfall(cm),3hr_New_Snowfall(cm),Cloud_Cover(1/10),Middle_Low_Cloud_Cover(1/10),Lowest_Cloud_Height(100m),Visibility(10m),30cm_Ground_Temperature(C),power,d2m,t2m,time,Temp_Dew_Point_Diff,log_aerosol,log_Prec(mm),log_Snowfall(cm),t2m_d2m_diff,power_lag_1,power_lag_2,power_lag_3,power_rolling_mean_24,power_rolling_std_24,day_of_week,is_weekend,Temp_Humidity_Interaction,Visibility_Cloud_Interaction
0,22.0,2013,3,20,17,3.9,0.0,72.0,-0.6,1.40,0.0,0.0,0.000000,0.000000,10.0,2500.000000,10.4,5356800.0,267.97684,273.97412,2013-03-20 17:00:00,4.5,3.135494,0.0,0.0,5.99728,4492800.0,4684800.0,4742400.0,3204400.0,2.686564e+06,2,0,-2.34,0.000000
1,33.0,2013,3,20,18,2.1,0.0,69.0,-3.0,0.66,0.0,0.0,1.000000,1.000000,10.0,2500.000000,10.4,2726400.0,267.81885,273.76404,2013-03-20 18:00:00,5.1,3.526361,0.0,0.0,5.94519,5356800.0,4492800.0,4684800.0,3317600.0,2.602003e+06,2,0,-6.30,2500.000000
2,40.0,2013,3,20,19,0.7,0.0,68.0,-4.5,0.07,0.0,0.0,0.923077,0.923077,10.0,2461.538462,10.3,201600.0,267.64540,273.56427,2013-03-20 19:00:00,5.2,3.713572,0.0,0.0,5.91887,2726400.0,5356800.0,4492800.0,3292000.0,2.630551e+06,2,0,-3.15,2272.189349
3,48.0,2013,3,21,7,-2.3,0.0,72.0,-6.6,0.01,0.0,0.0,0.000000,0.000000,10.0,2000.000000,8.3,28800.0,268.66763,279.48187,2013-03-21 07:00:00,4.3,3.891820,0.0,0.0,10.81424,201600.0,2726400.0,5356800.0,3172400.0,2.713142e+06,3,0,15.18,0.000000
4,38.0,2013,3,21,8,-0.6,0.0,60.0,-7.4,0.40,0.0,0.0,0.000000,0.000000,10.0,2000.000000,8.1,1468800.0,269.65826,279.11100,2013-03-21 08:00:00,6.8,3.663562,0.0,0.0,9.45274,28800.0,201600.0,2726400.0,3005600.0,2.688563e+06,3,0,4.44,0.000000


In [8]:
data = pd.concat([data.drop('power', axis=1), data['power']], axis=1)

data.head()

,aerosol,year,month,day,hour,Temp(C),Prec(mm),Humidity(%),Dew_Point(C),Solar_Radiation(MJ/m2),Snowfall(cm),3hr_New_Snowfall(cm),Cloud_Cover(1/10),Middle_Low_Cloud_Cover(1/10),Lowest_Cloud_Height(100m),Visibility(10m),30cm_Ground_Temperature(C),d2m,t2m,time,Temp_Dew_Point_Diff,log_aerosol,log_Prec(mm),log_Snowfall(cm),t2m_d2m_diff,power_lag_1,power_lag_2,power_lag_3,power_rolling_mean_24,power_rolling_std_24,day_of_week,is_weekend,Temp_Humidity_Interaction,Visibility_Cloud_Interaction,power
0,22.0,2013,3,20,17,3.9,0.0,72.0,-0.6,1.40,0.0,0.0,0.000000,0.000000,10.0,2500.000000,10.4,267.97684,273.97412,2013-03-20 17:00:00,4.5,3.135494,0.0,0.0,5.99728,4492800.0,4684800.0,4742400.0,3204400.0,2.686564e+06,2,0,-2.34,0.000000,5356800.0
1,33.0,2013,3,20,18,2.1,0.0,69.0,-3.0,0.66,0.0,0.0,1.000000,1.000000,10.0,2500.000000,10.4,267.81885,273.76404,2013-03-20 18:00:00,5.1,3.526361,0.0,0.0,5.94519,5356800.0,4492800.0,4684800.0,3317600.0,2.602003e+06,2,0,-6.30,2500.000000,2726400.0
2,40.0,2013,3,20,19,0.7,0.0,68.0,-4.5,0.07,0.0,0.0,0.923077,0.923077,10.0,2461.538462,10.3,267.64540,273.56427,2013-03-20 19:00:00,5.2,3.713572,0.0,0.0,5.91887,2726400.0,5356800.0,4492800.0,3292000.0,2.630551e+06,2,0,-3.15,2272.189349,201600.0
3,48.0,2013,3,21,7,-2.3,0.0,72.0,-6.6,0.01,0.0,0.0,0.000000,0.000000,10.0,2000.000000,8.3,268.66763,279.48187,2013-03-21 07:00:00,4.3,3.891820,0.0,0.0,10.81424,201600.0,2726400.0,5356800.0,3172400.0,2.713142e+06,3,0,15.18,0.000000,28800.0
4,38.0,2013,3,21,8,-0.6,0.0,60.0,-7.4,0.40,0.0,0.0,0.000000,0.000000,10.0,2000.000000,8.1,269.65826,279.11100,2013-03-21 08:00:00,6.8,3.663562,0.0,0.0,9.45274,28800.0,201600.0,2726400.0,3005600.0,2.688563e+06,3,0,4.44,0.000000,1468800.0


In [9]:
data.to_csv('done_preprocessing.csv', index=False)

In [10]:

# Selecting features and target
X = data.drop(columns=['power', 'time'])
y = data['power']

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:
date_cols = ['year', 'month', 'day', 'hour', 'day_of_week', 'is_weekend']

In [17]:
def compare_cols(cols):
    s=1
    for i in cols:
        print(f"feature is: {i}")
        train_unique_count = len(X_train[i].unique())
        test_unique_count = len(X_test[i].unique())
        print(f"# of train unique values is: {train_unique_count}")
        print(f"# of test unique values is: {test_unique_count}")
        all_values = set(X_train[i]).union(set(X_test[i]))
        D = pd.DataFrame({'Unique': list(all_values)})
        D['train_count'] = [len(X_train[X_train[i] == j]) for j in all_values]
        D['train_percent'] = (D['train_count']/ len(X_train)) * 100
        D['test_count'] = [len(X_test[X_test[i] == j]) for j in all_values]
        D['test_percent'] = (D['test_count']/ len(X_test)) * 100
        print(D.sort_values('train_count', ascending = False))
        print('--------------------')
        print(f"Total number of navigated features is: {s}")
        s+=1
        print('--------------------')

In [18]:
compare_cols(cols = date_cols)

feature is: year
# of train unique values is: 10
# of test unique values is: 10
   Unique  train_count  train_percent  test_count  test_percent
4    2017         3597      10.294791         881     10.085861
6    2019         3592      10.280481         865      9.902690
9    2022         3581      10.248998         889     10.177447
3    2016         3580      10.246136         878     10.051517
7    2020         3574      10.228964         896     10.257584
2    2015         3561      10.191757         861      9.856898
8    2021         3552      10.165999         912     10.440756
5    2018         3523      10.082999         890     10.188895
1    2014         3521      10.077275         917     10.497997
0    2013         2859       8.182599         746      8.540355
--------------------
Total number of navigated features is: 1
--------------------
feature is: month
# of train unique values is: 12
# of test unique values is: 12
    Unique  train_count  train_percent  test_count  

In [4]:

# Applying MinMaxScaler
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

# Scaling the features
X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Scaling the target variable
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1)).flatten()

print(X_train_scaled.shape, y_train_scaled.shape, X_test_scaled.shape, y_test_scaled.shape)